In [20]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm
import re
import time
from functools import partial
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import selenium.webdriver as webdriver
from selenium.webdriver.support.wait import WebDriverWait

In [21]:
main_url='http://www.yangcheon.go.kr/site/mayor/ex/bbs/List.do?cbIdx=414#'
stop_date='2020.12.01'

In [22]:
# 각각의 리스트 생성
date_list=[]
title_list=[]
text_list=[] 

In [23]:
def clean_text(text):
    """
    한글,영문,숫자 남기고 제거한다.
    """
    
    text = re.sub(r'[^ ㄱ-ㅣ가-힣A-Za-z]', '', text) #특수기호 제거, 정규 표현식    

    return text

In [24]:
options=webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('disable-gpu')



driver = webdriver.Chrome('./chromedriver',options=options)
driver2 = webdriver.Chrome('./chromedriver',options=options)


driver.implicitly_wait(10)
driver2.implicitly_wait(10)

In [25]:
driver.get(main_url)

In [26]:
next_page=3

In [27]:
def click_next(page):
    next_button=driver.find_element_by_xpath('//*[@id="bbsFVo"]/div[2]/div[2]/a['+str(page)+']')#3~12까지
    next_button.click()


In [28]:
def get_complain(page):
    x=driver.find_elements_by_css_selector('#bbsFVo > article.board-row > table > tbody > tr > td.subject > a')
    if x ==[]:
        click_next(page)
        page=page+1
        page=page%13
        if page==0:
            page=3
        get_complain(page)
    else:
        for i in x:
            i=int(i.get_attribute('onclick').split(',')[1][1:-1])
            driver2.get('http://www.yangcheon.go.kr/site/mayor/ex/bbs/View.do?cbIdx=414&bcIdx='+str(i)+'&parentSeq='+str(i))
            title=driver2.find_element_by_css_selector('#content-container > article > table > tbody > tr:nth-child(1) > td').text
            title=clean_text(title)
            text=driver2.find_element_by_css_selector('#content-container > article > table > tbody > tr:nth-child(2) > td > div > div').text
            text=clean_text(text)
            date=driver2.find_element_by_css_selector('#content-container > article > table > tbody > tr:nth-child(5) > td:nth-child(2)').text
            if date==stop_date:
                return
            date_list.append(date)
            title_list.append(title)
            text_list.append(text)
        click_next(page)
        page=page+1
        page=page%13
        if page==0:
            page=3

        get_complain(page)
        
    

In [29]:
get_complain(3)

In [30]:
driver.close()
driver2.close()

In [31]:
import pandas as pd

In [32]:
df=pd.DataFrame({"date":date_list,"title":title_list,"text":text_list}) 

In [33]:
df

,date,title,text
0,2020.12.28,국과수 거리 오솔길 공원 미끄럼,지난 월 이미 한차례 의견을 올린바 있습니다요즘 날씨가 춥고 습한날에는 오솔길 ...
1,2020.12.21,우리구청의 원칙은 없나요,수고 많으십니다아울러 확정적으로 구청주무관님이 확인하고 공사를 지시하여 아무런 이상...
2,2020.12.21,학원교습소 휴원지원금,수고하십니다 주간 강제휴원중인 교습소운영자입니다다들 코로나로 인해 어렵지만이번의 강...
3,2020.12.21,신월동 분리수거장 재설치해주시거나 다른 해결방안 제시해주시죠,무슨의도로 분리수거장을 철거했는지 전혀 이해가안되고 주민들에 의견은 반영이 된 결과...
4,2020.12.21,휴업지원금 재문의,답변 잘 받았습니다다른구들 노원구 성북구 강북구 등은 학원교습소 휴업지원금 만원씩 ...
5,2020.12.21,도로 상수도관 누수 결빙,도로축대 미터 밑에 있는 주택 경계 외벽에 월초부터 물이 계속나오고 있고한파에 결...
6,2020.12.20,양천구 신월동 쓰레기분리수거장 재설치 요청 목동과 분리요청,안녕하세요신월동에 거주중인 양천구민입니다무슨 생각으로 양천구와 산하 동들을 운영하시...
7,2020.12.16,질문에 대한 엉뚱한 답변 시정 바람,아침 시부터 단속 시작 한 것이 년정도 된 듯한데융통성 있게 단속해 달라고 게시글을...
8,2020.12.16,입장 큐알 설치,코로나로 인해 식당을 출입할때 방문자는 입장을 확인하는 기록을 남기게 되는데요식당 ...
9,2020.12.15,교습소학원 지원금요청,강동구는 교습소에 재난지원금 신청받던데양천구는 신청안받나요주간 집합금지라 경제적으로...
